In [1]:
import os
import librosa
import numpy as np
import skimage.io
import concurrent.futures

In [2]:
mp3_folder = "data/full_audio/30_sec_splits"
spectrogram_folder = 'data/full_audio/30_sec_splits/spectrograms'

def scale_minmax(X, min=0.0, max=255.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled.astype(np.uint8)

def process_file(file_name):
    try:
        # Load the MP3 file
        y, sr = librosa.load(file_name)
        
        # Create the spectrogram
        spect = librosa.feature.melspectrogram(y=y, sr=sr)
        spect = np.log(spect + 1e-9)
        spect = scale_minmax(spect)

        # Check if spectrogram contains invalid values
        if np.isnan(spect).any() or np.isinf(spect).any():
            raise ValueError("Spectrogram contains invalid values.")

        # Flip spectrogram vertically
        spect = np.flip(spect, axis=0)

        # Invert colors for better visualization
        spect = 255 - spect

        # Get genre and original filename
        genre, original_filename = os.path.split(file_name)
        genre = os.path.basename(genre)

        # Construct output directory
        output_folder = os.path.join(spectrogram_folder, genre)
        os.makedirs(output_folder, exist_ok=True)

        # Construct output filename
        output_filename = f"{original_filename[:-4]}.png"
        output_path = os.path.join(output_folder, output_filename)

        # Save spectrogram
        skimage.io.imsave(output_path, spect)

        # Global counter for tracking progress
        global c
        c += 1
        if c % 1000 == 0:
            print(f"Processed {c} files")

    except Exception as e:
        print(f"Error processing {file_name}: {e}")

c = 0
files = []
for root, dirs, files_in_dir in os.walk(mp3_folder):
    for file in files_in_dir:
        if file.endswith(".mp3"):
            files.append(os.path.join(root, file))

with concurrent.futures.ThreadPoolExecutor(max_workers=18) as executor:
    futures = [executor.submit(process_file, file_name) for file_name in files]
    concurrent.futures.wait(futures)